# Gemini & ChatGPT

In [ ]:
import os

print(f"REINSTALL: {os.environ.get('REINSTALL')}")
if os.environ["REINSTALL"] == "TRUE":
    !pip install -U langchain==0.3.0
    !pip install -U langchain_openai==0.2.1
    !pip install -U google-cloud-aiplatform==1.38
    !pip install -U google-cloud-aiplatform==1.69.0
    !pip install -U langchain-google-vertexai==2.0.3
    !pip install -U gradio==1.4.2

In [ ]:
# 2024/10/21 version

from google.colab import userdata
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
import json

api_key = userdata.get('openai-key')
os.environ['OPENAI_API_KEY'] = api_key
API_KEY = api_key

llm = ChatOpenAI(openai_api_key=API_KEY)

ANTHROPIC_API_KEY = userdata.get('anthropic-key')
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

llm = ChatAnthropic(model="claude-3-opus-20240229")

GOOGLE_API_KEY = userdata.get('jerry-argolis-api-key')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# setup the gemini pro
gemini_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

SERPER_API_KEY = userdata.get('google-serper-key')
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

def print_message(aiMessage, messageOnly=True):
    print("RESPONSE: {}".format(aiMessage.content))
    if messageOnly == False:
        print(json.dumps(aiMessage.additional_kwargs, indent=2))
        print(json.dumps(aiMessage.response_metadata, indent=2))
        print(json.dumps(aiMessage.usage_metadata, indent=2))

%env LANGCHAIN_TRACING_V2=true
%env LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith')
%env LANGCHAIN_PROJECT=langgraph-test

# https://smith.langchain.com